### 1. Setup and Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

### 2. Load Raw Data

In [3]:
ROOT = Path.cwd().parent.parent.parent

annual_path = ROOT / "data/raw/fundamentals/fmp/fundamentals_annual.parquet"
quarter_path = ROOT / "data/raw/fundamentals/fmp/fundamentals_quarter.parquet"

dfA = pd.read_parquet(annual_path)
dfQ = pd.read_parquet(quarter_path)

dfA.head(), dfQ.head()

(         date symbol       revenue   grossProfit  operatingIncome     netIncome    eps   totalAssets  totalStockholdersEquity  totalLiabilities     totalDebt  operatingCashFlow  freeCashFlow  \
 0  2017-01-29   NVDA  6.910000e+09  4.063000e+09     1.934000e+09  1.666000e+09  0.077  9.841000e+09             5.762000e+09      4.079000e+09  2.816000e+09       1.672000e+09  1.496000e+09   
 1  2018-01-28   NVDA  9.714000e+09  5.822000e+09     3.210000e+09  3.047000e+09  0.130  1.124100e+10             7.471000e+09      3.770000e+09  2.000000e+09       3.502000e+09  2.909000e+09   
 2  2019-01-27   NVDA  1.171600e+10  7.171000e+09     3.804000e+09  4.141000e+09  0.170  1.329200e+10             9.342000e+09      3.950000e+09  1.988000e+09       3.743000e+09  3.143000e+09   
 3  2020-01-26   NVDA  1.091800e+10  6.768000e+09     2.846000e+09  2.796000e+09  0.120  1.731500e+10             1.220400e+10      5.111000e+09  2.643000e+09       4.761000e+09  4.272000e+09   
 4  2021-01-31   NVDA  1.

### 3. Explore Raw Data

In [4]:
print("Annual shape:", dfA.shape)
print("Quarter shape:", dfQ.shape)

print("Annual years:", dfA['year'].unique())
print("Quarter years:", dfQ['year'].unique())

dfA['period'].value_counts(), dfQ['period'].value_counts()


Annual shape: (4144, 24)
Quarter shape: (5089, 24)
Annual years: [2017 2018 2019 2020 2021 2022 2023 2024 2025]
Quarter years: [2023 2024 2025 2022]


(period
 A    4144
 Name: count, dtype: int64,
 period
 Q    5089
 Name: count, dtype: int64)

In [5]:
print("Annual symbol count:", dfA['symbol'].nunique())
print("Quarter symbol count:", dfQ['symbol'].nunique())

Annual symbol count: 503
Quarter symbol count: 503


### 4. Normalize Year Column

In [6]:
dfA['year'] = pd.to_datetime(dfA['date'], errors='coerce').dt.year
dfQ['year'] = pd.to_datetime(dfQ['date'], errors='coerce').dt.year

### 5. Filter Data >= 2017

In [7]:
dfA = dfA[dfA['year'] >= 2017]
dfQ = dfQ[dfQ['year'] >= 2017]

dfA.shape, dfQ.shape


((4144, 24), (5089, 24))

### 6. Clean Quarterly: Pick Latest Quarter per Symbol-Year

In [8]:
dfQ_latest = (
    dfQ.sort_values(['symbol', 'year', 'date'])
       .groupby(['symbol', 'year'], as_index=False)
       .last()
)

dfQ_latest.head()

,symbol,year,date,revenue,grossProfit,operatingIncome,netIncome,eps,totalAssets,totalStockholdersEquity,totalLiabilities,totalDebt,operatingCashFlow,freeCashFlow,capitalExpenditure,numberOfShares,enterpriseValue,stockPrice,marketCapitalization,minusCashAndCashEquivalents,addTotalDebt,sharesOutstanding,marketCap,period
0,A,2023,2023-10-31,1.688000e+09,9.150000e+08,4.080000e+08,4.750000e+08,1.63,1.076300e+10,5.845000e+09,4.918000e+09,2.735000e+09,5.160000e+08,4.320000e+08,-8.400000e+07,2.920000e+08,3.132904e+10,103.37,3.018404e+10,1.590000e+09,2.735000e+09,2.920000e+08,3.018404e+10,Q
1,A,2024,2024-10-31,1.701000e+09,9.160000e+08,4.080000e+08,3.510000e+08,1.23,1.184600e+10,5.898000e+09,5.948000e+09,3.390000e+09,4.810000e+08,3.880000e+08,-9.300000e+07,2.900000e+08,3.985090e+10,130.31,3.778990e+10,1.329000e+09,3.390000e+09,2.900000e+08,3.778990e+10,Q
2,A,2025,2025-07-31,1.738000e+09,9.140000e+08,3.600000e+08,3.360000e+08,1.18,1.222600e+10,6.370000e+09,5.856000e+09,3.409000e+09,3.620000e+08,5.730000e+08,2.110000e+08,2.850000e+08,3.459485e+10,114.81,3.272085e+10,1.535000e+09,3.409000e+09,2.850000e+08,3.272085e+10,Q
3,AAPL,2023,2023-12-30,1.195750e+11,5.485500e+10,4.037300e+10,3.391600e+10,2.19,3.535140e+11,7.410000e+10,2.794140e+11,1.080400e+11,3.989500e+10,3.750300e+10,-2.392000e+09,1.550976e+10,3.053375e+12,192.53,2.986095e+12,4.076000e+10,1.080400e+11,1.550976e+10,2.986095e+12,Q
4,AAPL,2024,2024-12-28,1.243000e+11,5.827500e+10,4.283200e+10,3.633000e+10,2.41,3.440850e+11,6.675800e+10,2.773270e+11,9.679900e+10,2.993500e+10,2.699500e+10,-2.940000e+09,1.508172e+10,3.921238e+12,255.59,3.854738e+12,3.029900e+10,9.679900e+10,1.508172e+10,3.854738e+12,Q


### 7. Clean Annual: Pick Final Annual Filing per Symbol-Year

In [9]:
dfA_latest = (
    dfA.sort_values(['symbol', 'year', 'date'])
       .groupby(['symbol', 'year'], as_index=False)
       .last()
)

dfA_latest.head()


,symbol,year,date,revenue,grossProfit,operatingIncome,netIncome,eps,totalAssets,totalStockholdersEquity,totalLiabilities,totalDebt,operatingCashFlow,freeCashFlow,capitalExpenditure,numberOfShares,enterpriseValue,stockPrice,marketCapitalization,minusCashAndCashEquivalents,addTotalDebt,sharesOutstanding,marketCap,period
0,A,2017,2017-10-31,4.472000e+09,2.409000e+09,8.410000e+08,6.840000e+08,2.12,8.426000e+09,4.826000e+09,3.591000e+09,2.011000e+09,8.890000e+08,7.130000e+08,-176000000.0,322000000.0,2.123866e+10,68.03,2.190566e+10,2.678000e+09,2.011000e+09,322000000.0,2.190566e+10,A
1,A,2018,2018-10-31,4.914000e+09,2.687000e+09,9.040000e+08,3.160000e+08,0.98,8.541000e+09,4.567000e+09,3.970000e+09,1.799000e+09,1.087000e+09,9.100000e+08,-177000000.0,321000000.0,2.034959e+10,64.79,2.079759e+10,2.247000e+09,1.799000e+09,321000000.0,2.079759e+10,A
2,A,2019,2019-10-31,5.163000e+09,2.805000e+09,9.410000e+08,1.071000e+09,3.41,9.452000e+09,4.748000e+09,4.704000e+09,2.407000e+09,1.021000e+09,8.650000e+08,-156000000.0,314000000.0,2.481050e+10,75.75,2.378550e+10,1.382000e+09,2.407000e+09,314000000.0,2.378550e+10,A
3,A,2020,2020-10-31,5.339000e+09,2.837000e+09,8.460000e+08,7.190000e+08,2.33,9.627000e+09,4.873000e+09,4.754000e+09,2.359000e+09,9.210000e+08,8.020000e+08,-119000000.0,309000000.0,3.246381e+10,102.09,3.154581e+10,1.441000e+09,2.359000e+09,309000000.0,3.154581e+10,A
4,A,2021,2021-10-31,6.319000e+09,3.407000e+09,1.347000e+09,1.210000e+09,3.98,1.070500e+10,5.389000e+09,5.316000e+09,2.729000e+09,1.485000e+09,1.296000e+09,-189000000.0,304000000.0,4.899124e+10,157.06,4.774624e+10,1.484000e+09,2.729000e+09,304000000.0,4.774624e+10,A


### 8. Merge annual and quarter data (Annual priority)

In [10]:
dfC = pd.concat([dfA_latest, dfQ_latest], ignore_index=True)

df_clean = (
    dfC.sort_values(['symbol', 'year', 'period'])  # A before Q
       .groupby(['symbol', 'year'], as_index=False)
       .first()
)

df_clean.head(20), df_clean.shape

(   symbol  year        date       revenue   grossProfit  operatingIncome     netIncome   eps   totalAssets  totalStockholdersEquity  totalLiabilities     totalDebt  operatingCashFlow  freeCashFlow  \
 0       A  2017  2017-10-31  4.472000e+09  2.409000e+09     8.410000e+08  6.840000e+08  2.12  8.426000e+09             4.826000e+09      3.591000e+09  2.011000e+09       8.890000e+08  7.130000e+08   
 1       A  2018  2018-10-31  4.914000e+09  2.687000e+09     9.040000e+08  3.160000e+08  0.98  8.541000e+09             4.567000e+09      3.970000e+09  1.799000e+09       1.087000e+09  9.100000e+08   
 2       A  2019  2019-10-31  5.163000e+09  2.805000e+09     9.410000e+08  1.071000e+09  3.41  9.452000e+09             4.748000e+09      4.704000e+09  2.407000e+09       1.021000e+09  8.650000e+08   
 3       A  2020  2020-10-31  5.339000e+09  2.837000e+09     8.460000e+08  7.190000e+08  2.33  9.627000e+09             4.873000e+09      4.754000e+09  2.359000e+09       9.210000e+08  8.020000e+0

### 9. Transform wide format to long format

In [12]:
df_wide = df_clean.copy()
df_wide.columns.tolist()

['symbol',
 'year',
 'date',
 'revenue',
 'grossProfit',
 'operatingIncome',
 'netIncome',
 'eps',
 'totalAssets',
 'totalStockholdersEquity',
 'totalLiabilities',
 'totalDebt',
 'operatingCashFlow',
 'freeCashFlow',
 'capitalExpenditure',
 'numberOfShares',
 'enterpriseValue',
 'stockPrice',
 'marketCapitalization',
 'minusCashAndCashEquivalents',
 'addTotalDebt',
 'sharesOutstanding',
 'marketCap',
 'period']

In [17]:
metric_map = {
    "revenue" : "revenue",
    "grossProfit" : "gross_profit",
    "operatingIncome" : "operating_income",
    "netIncome" : "net_income",
    "eps" : "eps",
    "totalAssets" : "total_assets",
    "totalStockholdersEquity" : "total_stockholders_equity",
    "totalLiabilities" : "total_liabilities",
    "totalDebt" : "total_debt",
    "operatingCashFlow" : "operating_cash_flow",
    "freeCashFlow" : "free_cash_flow",
    "capitalExpenditure" : "capital_expenditure",
    "numberOfShares" : "number_of_shares",
    "enterpriseValue" : "enterprise_value",
    "marketCapitalization" : "market_capitalization",
    "sharesOutstanding" : "shares_outstanding",
}

value_cols = list(metric_map.keys())

# =============================================
# Period_end at long format
# =============================================

df_long = df_wide.copy()
df_long["period_end"] = pd.to_datetime(df_long["date"]).dt.date

df_long["period_type"] = df_long["period"].map({
    "A" : "annual",
    "Q" : "quarterly"
})

if df_long["period_type"].isna().any():
    raise ValueError("Some rows have invalid period value (not A/Q).")

# =============================================
# Melt to long format
# =============================================

df_long = df_long.melt(
    id_vars = ["symbol", "period_end", "period_type"],
    value_vars = value_cols,
    var_name = "metric_raw",
    value_name = "value"
)

df_long["metric"] = df_long["metric_raw"].map(metric_map)
df_long = df_long.dropna(subset=["value"])

print("Long format rows:", len(df_long))
df_long.head(10)

Long format rows: 72001


,symbol,period_end,period_type,metric_raw,value,metric
0,A,2017-10-31,annual,revenue,4.472000e+09,revenue
1,A,2018-10-31,annual,revenue,4.914000e+09,revenue
2,A,2019-10-31,annual,revenue,5.163000e+09,revenue
3,A,2020-10-31,annual,revenue,5.339000e+09,revenue
4,A,2021-10-31,annual,revenue,6.319000e+09,revenue
5,A,2022-10-31,annual,revenue,6.848000e+09,revenue
6,A,2023-10-31,annual,revenue,6.833000e+09,revenue
7,A,2024-10-31,annual,revenue,6.510000e+09,revenue
8,A,2025-07-31,quarterly,revenue,1.738000e+09,revenue
9,AAPL,2017-09-30,annual,revenue,2.292340e+11,revenue


In [18]:
# =============================================
# Final tidy dataframe
# =============================================

df_out = df_long[[
    "symbol",
    "period_end",
    "period_type",
    "metric",
    "value"
]]

# =============================================
# Save to parquet
# =============================================

OUTPUT_DIR = ROOT / "data/curated/fundamentals/"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
df_out.to_parquet(OUTPUT_DIR / "fundamentals_clean.parquet", index=False)
print("Saved cleaned fundamentals to:", OUTPUT_DIR / "fundamentals_clean.parquet")

Saved cleaned fundamentals to: /home/clsx6609/ds5110/data/curated/fundamentals/fundamentals_clean.parquet
